In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('novel_story_sep.csv', lineterminator='\n')

In [3]:
df.columns

Index(['itemId', 'title', 'story', 'review', 'piece'], dtype='object')

# LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습시키기

In [4]:
import gensim
gensim.__version__

'3.8.1'

In [5]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet

In [6]:
stories = df['story'] + df['review'] + df['piece']

In [7]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [8]:
id2word = corpora.Dictionary(data_word)

In [9]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, random_seed=12, id2word=id2word)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print(coherence_ldamallet) #응집성 지수

0.39875036551034687


In [10]:
# Select the model and print the topics
# optimal_model = model_list[model_list_num]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics()[5])

(5, '0.024*"시간" + 0.018*"생각" + 0.011*"나" + 0.010*"세상" + 0.010*"기억" + 0.008*"일" + 0.007*"너" + 0.006*"본문" + 0.006*"선택" + 0.006*"왜"')


In [11]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=20)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data['itemId'],Data['title'],Data['story'], sent_topics_df], axis=1)

    return(sent_topics_df)

In [12]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df)

<class 'pandas.core.frame.DataFrame'>


In [13]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [14]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [15]:
sent_topics_sorteddf_mallet[['Perc_Contribution', 'Dominant_Topic', 'Topic_Keywords']]

,Perc_Contribution,Dominant_Topic,Topic_Keywords
801,0.5967,0.0,"사건, 미스터리, 남자, 살인, 진실, 당신, 시작, 스릴러, 경찰, 형사, 범인,..."
479,0.9438,1.0,"nan, 조선, 왕, 여인, 사랑, 드라마, 로맨스, 황, 마음, 꽃, 역사, 시작..."
1079,0.8379,2.0,"세계, 나, 존재, 한국, 여성, 집, 사랑, 우주, 공간, 젊은, 방식, 삶, 기..."
942,0.5284,3.0,"아이, 엄마, 친구, 청소년, 아빠, 학교, 고민, 주인공, 가족, 소녀, 나, 어..."
895,0.8347,4.0,"한국, 사회, 박완서, 삶, 언어, 글, 결정, 판, 대표, 어머니, 새로운, 다양..."
510,0.7154,5.0,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
1056,0.6530,6.0,"인간, 이상, 존재, 사회, 일, 영화, 주인공, 사실, 나, 세계, 소재, 배, ..."
843,0.7758,7.0,"그녀, 눈, 얼굴, 손, 소리, 몸, 입, 머리, 방, 거리, 향, 명, 달, 누구..."
263,0.5288,8.0,"삶, 죽음, 아버지, 살, 세상, 쪽, 인간, 길, 인생, 존재, 아들, 의미, 마..."
483,0.6867,9.0,"꿈, 면, 소년, 마음, 밤, 새로운, 손님, 감동, 모습, 세상, 음악, 판타지,..."


In [16]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0.0      79
1.0     502
2.0      59
3.0      66
4.0      25
5.0      41
6.0      52
7.0      41
8.0      67
9.0      40
10.0     85
11.0     61
dtype: int64

In [17]:
topic_1 = df_topic_story[df_topic_story['Dominant_Topic']==1]
topic_2 = df_topic_story[df_topic_story['Dominant_Topic']==5]
topic_2[topic_2['Perc_Contribution'] > 0.0]

,itemId,title,story,Dominant_Topic,Perc_Contribution,Topic_Keywords
22,210123203,시간을 파는 상점,시간 흐름 숨겨진 마법 비밀 자음 모음 청소년 수상한 김선영 시간 파는 상점 심사 ...,5.0,0.3927,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
30,354296981,벼랑 위의 집,어른 아이 러블리 판타지 다자 기만 집 찾아가는 사랑스러운 여정 완벽하다 평 받은 ...,5.0,0.1995,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
35,327002623,독고솜에게 반하면,사람 기회 쉽게 포기 건 아닐까 소문 편견 첫인상 속단 장벽 너머 걸음 다가가는 용...,5.0,0.2350,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
56,283955925,구미호 식당,죽었다는 사실 무서운건 되돌릴 시간 당신 일주일 시간 없다면 시간 한번 지나면 다시...,5.0,0.6951,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
63,354595639,시간을 파는 상점(10주년 특별판),읽은 시간 파는 상점 주년 특별판 시간 기다려주지 않는다면 시간 먼저 손 쥐 세상 ...,5.0,0.3773,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
77,290441582,산책을 듣는 시간,소리 듣지 해도 친구 미래 고민 누군가 좋아하는 평범한 십 소녀 수지 전하 진심 사...,5.0,0.2975,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
97,214106438,오즈의 의류 수거함,외 슬픈 소외 사람 밤 리다 자음 모음 청소년 오즈 의류 수거 함 서울 밤 세계 배...,5.0,0.2330,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
115,354470283,구미호 식당 3: 약속 식당,줄거리 보육원 만난 설 지키려 싸움 뛰어들었다가 죽은 채우 인간 태어날 날 기다리던...,5.0,0.6557,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
126,332958393,기억 1,기억 문 열리고 모험 시작 한국 사랑 베르나르 베르베르 기억 프랑스 전문 번역가 미...,5.0,0.2245,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."
136,316842227,시간을 파는 상점 2: 너를 위한 시간,시간 정말 사고팔 없을까 자음 모음 청소년 수상한 김선영 시간 파는 상점 너 시간 ...,5.0,0.4219,"시간, 생각, 나, 세상, 기억, 일, 너, 본문, 선택, 왜, 누구, 수도, 누군..."


In [18]:
df_topic_sort[df_topic_sort['Perc_Contribution'] > 0.5]

,itemId,title,story,Dominant_Topic,Perc_Contribution,Topic_Keywords
801,239589818,차일드 44 2: 시크릿 스피치,세계 흔든 전설 미스터리 스릴러 리들리 스콧 제작 톰 하디 게리 올드 주연 영화 차...,0.0,0.5967,"사건, 미스터리, 남자, 살인, 진실, 당신, 시작, 스릴러, 경찰, 형사, 범인,..."
645,255481804,지옥이 새겨진 소녀,단테 신곡 새기고 나타난 소녀 단테 신곡 모티브 안드레아스 그루 버 스릴러 결정 판...,0.0,0.5258,"사건, 미스터리, 남자, 살인, 진실, 당신, 시작, 스릴러, 경찰, 형사, 범인,..."
802,239590377,차일드 44 3: 에이전트 6,세계 흔든 전설 미스터리 스릴러 리들리 스콧 제작 톰 하디 게리 올드 주연 영화 차...,0.0,0.5966,"사건, 미스터리, 남자, 살인, 진실, 당신, 시작, 스릴러, 경찰, 형사, 범인,..."
343,354452028,메이든스,비밀 자기 자신 사일런트 페이 션트 화려하게 알렉스 마이 클리 디스 최신 화 제작 ...,0.0,0.5026,"사건, 미스터리, 남자, 살인, 진실, 당신, 시작, 스릴러, 경찰, 형사, 범인,..."
382,237256642,구르미 그린 달빛 1: 눈썹달,웹소설 전설 윤이수 궁 로맨스 윤이수 구르 그린 달빛 눈썹달 박보검 김유정 주연 월...,1.0,0.9243,"nan, 조선, 왕, 여인, 사랑, 드라마, 로맨스, 황, 마음, 꽃, 역사, 시작..."
...,...,...,...,...,...,...
87,354601636,위저드 베이커리,당신 감고 시간 있습니까 판매 위험한 소원 이루어지는 위저드 베이커리 사로잡은 병모...,9.0,0.6289,"꿈, 면, 소년, 마음, 밤, 새로운, 손님, 감동, 모습, 세상, 음악, 판타지,..."
803,241302973,세월의 돌 3: 봄의 대륙을 가로질러,룬 아이 윈터 러 룬 아이 데모 닉 발표 한국 대표 장르 문학가 인정받고 전민희 세...,9.0,0.6143,"꿈, 면, 소년, 마음, 밤, 새로운, 손님, 감동, 모습, 세상, 음악, 판타지,..."
847,244050583,도쿠가와 이에야스(1부) 1: 출생의 비밀,야마 오카 하치 대하 도쿠가 야스 출생 비밀 동란 기 영웅 호걸 천하 제패 꿈 만천...,11.0,0.5016,"세계, 인간, 역사, 시대, 미국, 일본, 전쟁, 세기, 프랑스, 쪽, 생각, 주의..."
135,284224211,고양이 1,고양이 눈 바라본 인간 미래 베르나르 베르베르 고양이 파리 살 고양이 바스테트 시각...,11.0,0.5986,"세계, 인간, 역사, 시대, 미국, 일본, 전쟁, 세기, 프랑스, 쪽, 생각, 주의..."
